In [1]:
import ollama
import gradio as gr

In [2]:
MODEL = "llama3.2"

In [29]:
system_prompt = "You are EventAIde, an efficient AI chatbot that assists users in finding\
events happening in cities they mention. Do not include unnecessary information or explanations.\
Always maintain a friendly, helpful, and professional tone."

system_message = [{"role":"system", "content": system_prompt}]

In [30]:
def chatbot(message, history):
    messages = system_message+history+[{"role":"user", "content":message}]
    response = ollama.chat(model = MODEL, messages=messages, stream=True)
    result = ""
    for chunk in response:
        result += chunk["message"]["content"] or ""
        yield result

In [31]:
gr.ChatInterface(fn = chatbot, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
